In [49]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [31]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Saint-Joseph,-21.3667,55.6167,77.00,78,0,1.14,RE,2021-03-03 23:37:08
1,1,Imeni Zhelyabova,58.9544,36.5956,31.64,99,100,10.16,RU,2021-03-03 23:38:24
2,2,Ushuaia,-54.8000,-68.3000,55.40,50,20,4.61,AR,2021-03-03 23:37:02
3,3,Salinopolis,-0.6136,-47.3561,78.12,87,100,10.45,BR,2021-03-03 23:38:25
4,4,Juneau,58.3019,-134.4197,39.20,60,90,6.91,US,2021-03-03 23:35:33


In [32]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [33]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
city_data_df["Max Temp"] = [temp if temp > 0 else 0 for temp in city_data_df["Max Temp"]]
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))


In [35]:
gmaps.figure()


Figure(layout=FigureLayout(height='420px'))

In [36]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 25
What is the maximum temperature you would like for your trip? 105


In [39]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Saint-Joseph,-21.3667,55.6167,77.00,78,0,1.14,RE,2021-03-03 23:37:08
1,1,Imeni Zhelyabova,58.9544,36.5956,31.64,99,100,10.16,RU,2021-03-03 23:38:24
2,2,Ushuaia,-54.8000,-68.3000,55.40,50,20,4.61,AR,2021-03-03 23:37:02
3,3,Salinopolis,-0.6136,-47.3561,78.12,87,100,10.45,BR,2021-03-03 23:38:25
4,4,Juneau,58.3019,-134.4197,39.20,60,90,6.91,US,2021-03-03 23:35:33
5,5,Rikitea,-23.1203,-134.9692,79.66,79,42,10.49,PF,2021-03-03 23:32:55
6,6,Butaritari,3.0707,172.7902,81.34,82,2,15.68,KI,2021-03-03 23:37:17
7,7,Olafsvik,64.8945,-23.7142,40.48,92,93,17.58,IS,2021-03-03 23:38:26
8,8,Hobart,-42.8794,147.3294,71.60,43,75,20.71,AU,2021-03-03 23:32:46
10,10,Xinmin,41.9908,122.8253,39.20,52,0,11.18,CN,2021-03-03 23:38:27


In [40]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Saint-Joseph,RE,77.00,-21.3667,55.6167,
1,Imeni Zhelyabova,RU,31.64,58.9544,36.5956,
2,Ushuaia,AR,55.40,-54.8000,-68.3000,
3,Salinopolis,BR,78.12,-0.6136,-47.3561,
4,Juneau,US,39.20,58.3019,-134.4197,
5,Rikitea,PF,79.66,-23.1203,-134.9692,
6,Butaritari,KI,81.34,3.0707,172.7902,
7,Olafsvik,IS,40.48,64.8945,-23.7142,
8,Hobart,AU,71.60,-42.8794,147.3294,
10,Xinmin,CN,39.20,41.9908,122.8253,


In [41]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [42]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [43]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [46]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [48]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))